In [ ]:
import pip
def install(package):
   pip.main(['install', package])

install('BeautifulSoup4')
install('webhoseio')
install('textblob')
install('nltk')

In [ ]:
#importing required packages
import webhoseio
import requests
import numpy as np
import pandas as pd
import json
import os
import time
import datetime

In [ ]:
completedf=pd.DataFrame()

In [ ]:
#getting current working directory
current_directory=os.getcwd()
root_directory=os.path.abspath(os.path.join(current_directory, os.pardir)) #To get the parent directory of current directory
#parent_directory=os.path.abspath(os.path.join(root_directory, os.pardir))
print(root_directory)

In [ ]:
path=root_directory+"/Data/Input Data/"

In [ ]:
#Reading all input JSON files and storing in 'completedf' DataFrame
for directory, subdirectory, filenames in os.walk(path):
    for filename in filenames:
        with open(os.path.join(directory, filename), "r") as f:
            df=pd.read_json(f)
            completedf=pd.concat([completedf,df],axis=0)
#completedf now has 15 columns

In [ ]:
#Appending tags from inner JSON (thread and entities) to completedf DataFrame
completedf["site_full"]=completedf["thread"].apply(lambda x: x["site_full"])
completedf["main_image"]=completedf["thread"].apply(lambda x: x["main_image"])
completedf["site_section"]=completedf["thread"].apply(lambda x: x["site_section"])
completedf["section_title"]=completedf["thread"].apply(lambda x: x["section_title"])
completedf["country"]=completedf["thread"].apply(lambda x: x["country"])
completedf["domain_rank"]=completedf["thread"].apply(lambda x: x["domain_rank"])
completedf["performance_score"]=completedf["thread"].apply(lambda x: x["performance_score"])
completedf["site"]=completedf["thread"].apply(lambda x: x["site"])
completedf["participants_count"]=completedf["thread"].apply(lambda x: x["participants_count"])
completedf["title_full"]=completedf["thread"].apply(lambda x: x["title_full"])
completedf["spam_score"]=completedf["thread"].apply(lambda x: x["spam_score"])
completedf["site_type"]=completedf["thread"].apply(lambda x: x["site_type"])
completedf["replies_count"]=completedf["thread"].apply(lambda x: x["replies_count"])

completedf["persons"]=completedf["entities"].apply(lambda x: x["persons"])
completedf["locations"]=completedf["entities"].apply(lambda x: x["locations"])
completedf["organizations"]=completedf["entities"].apply(lambda x: x["organizations"])

#complete df now has 31 columns

In [ ]:
#Creating processeddf DataFrame by removing redundant columns
processeddf = completedf.drop(["thread","entities"], 1)
#'processeddf' DataFrame has 29 columns

In [ ]:
#Finding missing values in processeddf
def missing_values_table(df):
       mis_val = df.isnull().sum()
       mis_val_percent = 100 * df.isnull().sum()/len(df)
       mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
       mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
       return mis_val_table_ren_columns

In [ ]:
missing_values_table(processeddf).sort_values("Missing Values",ascending=False).head(10)

In [ ]:
processeddf['main_image'] = processeddf['main_image'].fillna('Not Available')

In [ ]:
processeddf.replace({'\n': ' '}, regex=True)

In [ ]:
#converting 'published' variable to adtetime format and extracting day and saving it into new variable
processeddf["published"]=pd.to_datetime(processeddf['published'])
processeddf["published_day"]=processeddf["published"].dt.day

In [ ]:
#Check if a directory exists in the path or not;if not create one
def dir(file_path,directory_name):
    if not os.path.exists(file_path+"/"+directory_name):
        os.mkdir(file_path+"/"+directory_name)
    return file_path+"/"+directory_name

In [ ]:
#final_directory=dir(root_directory,"Final") #Create Final folder in root_directory
data_directory=dir(root_directory,"Data") #Create Data folder in final_directory
file_directory=dir(data_directory,"Processed Data") #Create Processed Data folder in data_directory

In [ ]:
processeddf.to_csv(file_directory+"/PocessedData.csv")